In [1]:
from starknet_py.net import AccountClient, KeyPair
from starknet_py.net.gateway_client import GatewayClient
from starknet_py.net.models import StarknetChainId
from starknet_py.net.signer.stark_curve_signer import StarkCurveSigner
from starknet_py.contract import Contract

import json
from dotenv import load_dotenv
load_dotenv()
import os

## Contracts address and abi

In [2]:
from constants import contract_address

with open('abis/polynomial_lr.json', 'r') as _file:
    contract_abi = json.load(_file)

## Setup account and contract

In [12]:
net = "testnet"
chain_id = StarknetChainId.TESTNET
client = GatewayClient(net=net)

# Access your .env file
private_key = int(os.getenv('PRIVATE_KEY'))
account_address = os.getenv('ACCOUNT_ADDRESS')

# account
key_pair = KeyPair.from_private_key(key=private_key)
signer = StarkCurveSigner(account_address, key_pair, StarknetChainId.TESTNET)
account_client = AccountClient(client=client, address=account_address, signer=signer, supported_tx_version=1)

# contract
contract = Contract(address=contract_address, abi=contract_abi, client=account_client)
# Contract.from_address makes additional request to fetch the ABI
# contract = await Contract.from_address(client=account_client, address=address)

## Commit model

In [7]:
import pandas as pd
from utils import pedersen_hash_chain

model_file = 'model/model.txt'
weight = pd.read_csv(model_file)

In [14]:
# Prepare pedersen hash

commitment = merkle_root([merkle_root(X_test), merkle_root(X_test)])
commit = await contract.functions['commit_test_data'].invoke(commitment, max_fee=int(1e18))